## cVaR

$X$ is a standard (mean 0 , variance 1 ) normal distribution and $Y$ is a Weibull distribution with cdf $$F(x)=\exp (x-1)$$ for $-\infty \leq x \leq 1$. For what $p>0.65$ does $cVaR_X(p)=cVaR_Y(p)$ ? you will need a calculation engine or a little Python code.

In [1]:
from scipy.stats import norm
from scipy.optimize import fsolve
import numpy as np
def equation(p):
    # VaR for standard normal
    var_X = norm.ppf(p)
    # VaR for the given Weibull distribution
    var_Y = 1 + np.log(p)
    return var_X - var_Y

# Use fsolve to solve for p
p_initial_guess = 0.7
p_solution = fsolve(equation, p_initial_guess)

print(f"The value of p for which VaR_X(p) = VaR_Y(p) is: {p_solution[0]}")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
The value of p for which VaR_X(p) = VaR_Y(p) is: 0.76989387870113


In [2]:
from scipy.stats import norm
from scipy.integrate import quad
import numpy as np

def cvar_X(p):
    return -norm.ppf(p)

def cvar_Y(p):
    var_Y = 1 + np.log(p)
    integrand = lambda x: x * np.exp(x - 1)
    integral, _ = quad(integrand, -np.inf, var_Y)
    return integral/p

def equation(p):
    return cvar_X(p) - cvar_Y(p)

# Use fsolve to solve for p
p_initial_guess = 0.7
p_solution = fsolve(equation, p_initial_guess)

print(f"The value of p for which cVaR_X(p) = cVaR_Y(p) is: {p_solution[0]}")

The value of p for which cVaR_X(p) = cVaR_Y(p) is: 0.6607191029073377


In [3]:
import numpy as np
from scipy.stats import norm
from scipy.integrate import quad

def cVaR_X(p):
    var_x = norm.ppf(1-p)
    integral, _ = quad(lambda x: x * norm.pdf(x), var_x, np.inf)
    return integral / (1 - p)

def cVaR_Y(p):
    var_y = np.log(1-p) + 1
    integral, _ = quad(lambda x: x * np.exp(x - 1), var_y, 1)
    return integral / (1 - p)

# Search for p
for p in np.linspace(0.65, 1, 10000):
    if abs(cVaR_X(p) - cVaR_Y(p)) < 1e-4:
        print(p)
        break

0.6587508750875087


In [4]:
from scipy.stats import norm
from scipy.optimize import fsolve
import numpy as np

def equation(p):
    # VaR for standard normal
    var_X = norm.ppf(1-p)
    
    # Check to avoid log(0)
    if 1-p <= 0.000001:
        return 1e10
    
    # VaR for the given Weibull distribution
    var_Y = 1 + np.log(1-p)
    return var_X - var_Y

# Use fsolve to solve for p
p_initial_guess = 0.8
p_solution = fsolve(equation, p_initial_guess)

print(f"The value of p for which VaR_X(1-p) = VaR_Y(1-p) is: {p_solution[0]}")


The value of p for which VaR_X(1-p) = VaR_Y(1-p) is: 0.8949429530714774


## Capital Buffer 

Suppose that your trading desk invests 1 billion each month in the equal-weighted portfolio of the 11 companies in ratio_data.xlsx. Further suppose that the current time is December 31 of the year that ends one year before the last year in ratio_data.xlsx. As the risk manager for this desk, you must specify at the current time a buffer amount of money to set aside that will equal the worst monthly loss in the coming year. You will be fired if you set aside too much or too little by more than 10 million. Based on the metrics discussed in Chapter 2, how much should you set aside at the current time to maximize your chances of keeping your job? Would you have kept your job at the end of the most recent year?

In [5]:
import pandas as pd
data = pd.read_excel('ratio_data.xlsx')

In [6]:
data

,Date,AAPL,AMZN,ED,F,JNJ,JPM,ORCL,TSLA,V,WMT,XOM
0,2021-12-31,1.074229,0.950748,1.098918,1.082335,1.097095,0.996978,0.961098,0.923145,1.118388,1.033013,1.022560
1,2021-11-30,1.105082,1.039924,1.039940,1.129194,0.963601,0.934899,0.945799,1.027612,0.916613,0.941173,0.940658
2,2021-10-29,1.058657,1.026602,1.038711,1.206215,1.008545,1.044131,1.104944,1.436530,0.950707,1.072033,1.096056
3,2021-09-30,0.931963,0.946482,0.962094,1.086723,0.932825,1.023382,0.977561,1.054042,0.972283,0.941121,1.078870
4,2021-08-31,1.042489,1.043034,1.033005,0.934050,1.011375,1.053828,1.022837,1.070605,0.931094,1.042757,0.961355
...,...,...,...,...,...,...,...,...,...,...,...,...
115,2012-05-31,0.989297,0.918111,1.025682,0.936170,0.968322,0.771289,0.900340,0.890432,0.938508,1.124879,0.916976
116,2012-04-30,0.974031,1.145129,1.017631,0.907759,0.986962,0.940873,1.010281,0.889635,1.042203,0.962582,0.995503
117,2012-03-30,1.105283,1.126996,1.005508,1.008077,1.013522,1.171764,0.996923,1.114636,1.014007,1.042817,1.002659
118,2012-02-29,1.188311,0.924141,0.995600,0.996780,0.996129,1.052011,1.036866,1.149295,1.158512,0.962842,1.038613


### Computing Metrics:

In [7]:
# Exclude non-numeric columns when computing the average
numeric_columns = data.select_dtypes(include=['number']).columns
data['avg_return'] = data[numeric_columns].mean(axis=1)

mean_return = data['avg_return'].mean()
std_dev_return = data['avg_return'].std()

### Project Potential Losses:

In [8]:
from scipy.stats import norm
confidence_level = 0.95
var_99 = mean_return - norm.ppf(confidence_level) * std_dev_return #Parametric VaR calculation
buffer_amount = -1 * var_99 * 1e9  # Since I invest 1 billion each month

In [9]:
print(std_dev_return)

0.041789429478060254


### Comparison with Actual Worst Monthly Loss:

In [10]:
# Convert the 'Date' column to a datetime format and set as the index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# For the comparison with actual worst monthly loss:
most_recent_year = data.index.year.max() - 1  # The year before the last year in the data
most_recent_year_data = data[data.index.year == most_recent_year]
worst_loss = most_recent_year_data['avg_return'].min() * 1e9
job_kept = (buffer_amount + 10e6) >= worst_loss >= (buffer_amount - 10e6)

# Printing the results
print(f"Buffer Amount to Set Aside: ${buffer_amount:,.2f}")
print(f"Would you have kept your job? {'Yes' if job_kept else 'No'}")

Buffer Amount to Set Aside: $-950,092,897.93
Would you have kept your job? No
